## Create new magentic fields with [magtense](https://github.com/cmt-dtu-energy/MagTense/tree/master)

In [1]:
from pmdd.utils.data_utils import create_db_mp

db_kwargs = {
    'n_samples': 10,
    'res': [32,32],
    'z_comp': False
}
create_db_mp('magfield_symm', n_workers=15, **db_kwargs)

100%|██████████| 1/1 [00:00<00:00, 21.98it/s]


/home/spol/Documents/repos/mag-diffusion/src/pmdd/utils/../../../data/magfield_symm_32_z_5_6.h5
/home/spol/Documents/repos/mag-diffusion/src/pmdd/utils/../../../data/magfield_symm_32_z_1_2.h5
/home/spol/Documents/repos/mag-diffusion/src/pmdd/utils/../../../data/magfield_symm_32_z_6_7.h5
/home/spol/Documents/repos/mag-diffusion/src/pmdd/utils/../../../data/magfield_symm_32_z_8_9.h5
/home/spol/Documents/repos/mag-diffusion/src/pmdd/utils/../../../data/magfield_symm_32_z_4_5.h5
/home/spol/Documents/repos/mag-diffusion/src/pmdd/utils/../../../data/magfield_symm_32_z_2_3.h5
/home/spol/Documents/repos/mag-diffusion/src/pmdd/utils/../../../data/magfield_symm_32_z_3_4.h5
/home/spol/Documents/repos/mag-diffusion/src/pmdd/utils/../../../data/magfield_symm_32_z_0_1.h5
/home/spol/Documents/repos/mag-diffusion/src/pmdd/utils/../../../data/magfield_symm_32_z_7_8.h5
/home/spol/Documents/repos/mag-diffusion/src/pmdd/utils/../../../data/magfield_symm_32_z_9_10.h5
Database created
